In [1]:
#IMPORT PACKAGES
import os.path
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import matplotlib.pyplot as plt

# from tabulate import tabulate

import json  
import copy
import laspy

from sklearn.cluster import DBSCAN
from PIL import Image

from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context
import utils as utl
import utils.t8_utils as t8


# Grounding DINO
from groundingdino.util.inference import  annotate, load_image, predict


# segment anything
# from segment_anything import build_sam, SamPredictor 


# diffusers
import torch
from torch.cuda.amp import autocast

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [4]:
#paths
path=Path(os.getcwd()).parents[0]

print(path)
input_folder_t4=path/'data'/'t4'/'test' 
input_folder_t6=path/'data'/'t6'/'test'
class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t8'/ 'test'
os.makedirs(output_folder, exist_ok=True)

#parameters
image_resolution = 0.01

t_score = 0.3

TEXT_PROMPT = "Door"

/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024


Import Classes

In [5]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 6, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


## Processing

In [6]:
point_cloud_files=utl.get_list_of_files(input_folder_t4,'.laz')
wall_files=utl.get_list_of_files(input_folder_t6,'.ttl')

In [7]:
for f_pcd in point_cloud_files:
    
    ckpt_repo_id = "ShilongLiu/GroundingDINO" 
    ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
    ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"
    device="cuda:1"
    
    with autocast():
        groundingdino_model = t8.load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename, device=device)
        
        
    reform_name='_'.join(ut.get_filename(f_pcd).split('_')[:4])
    for f_rdf in wall_files:
        if reform_name == '_'.join(ut.get_filename(f_rdf).split('_')[:4]):
            break
    print(f_pcd)
    print(f_rdf)
    doorNodes = []
    door_count = 0
    print(f'processing {ut.get_filename(f_pcd)}...') 
    
    wallNodes = t8.graph_to_wallnodes(f_pcd, f_rdf)
    
    t8.create_orthos(wallNodes, image_resolution = 0.01, octree_size = 10, fill_region = 12)
    
    t8.object_detection(wallNodes, model = groundingdino_model, TEXT_PROMPT = "Door") 
    
    doorNodes = t8.process_detections(wallNodes, t_score = 0.5, image_resolution = 0.01)
    t8.create_geometry(doorNodes)
            
    nodes = t8.merge_door_nodes(doorNodes)
    new_nodes = t8.filter_door_nodes(doorNodes = nodes, min_width = 0.5, max_width = 1.2, min_height = 1.5, max_height = 2.5, max_distance_to_ground = 0.5)
    
    t8.create_geometry(new_nodes)

    for doornode in new_nodes:
        print("%s : Door Score: %s => Referencelevel: %s; Width: %s; height: %s" %(doornode.name, doornode.doornessScore, doornode.reference_level, doornode.doorWidth, doornode.height))    
    
    print(f"Number of doors: {len(new_nodes)}")
    t8.export_door_nodes(doorNodes = new_nodes, output_folder = output_folder, filename = ut.get_filename(f_pcd))
    

final text_encoder_type: bert-base-uncased
Model loaded from /home/sdegeyter/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])
/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t4/test/25_Parking_01_F2_small_pred.laz
/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t6/test/25_Parking_01_F2_walls.ttl
processing 25_Parking_01_F2_small_pred...
18 wallNodes detected!
Matched 18 pointclouds to nodes


2024-05-31 16:08:31.412605: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 16:08:32.822718: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdegeyter/.conda/envs/pointcept/lib/python3.10/site-packages/cv2/../../lib64:
2024-05-31 16:08:32.822825: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdegeyter/.conda/envs/pointcept/lib/pyth

Doors_4 : Door Score: 0.6158147001784781 => Referencelevel: 0.01; Width: 1.17; height: 2.08
Doors_3 : Door Score: 0.6615254315474761 => Referencelevel: 0.02; Width: 1.02; height: 2.09
Doors_6 : Door Score: 0.7119081344293511 => Referencelevel: 0.05; Width: 0.92; height: 2.1
Doors_2 : Door Score: 0.53901416586793 => Referencelevel: 0.03; Width: 0.8; height: 2.11
Number of doors: 4
JSON data written to file: /home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t8/test/25_Parking_01_F2_doors.json
final text_encoder_type: bert-base-uncased
Model loaded from /home/sdegeyter/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])
/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t4/test/08_ShortOffice_01_F2_small_pred.laz
/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t6/test/08_ShortOffice_01_F2_walls.ttl


Doors_30 : Door Score: 0.6593811416366826 => Referencelevel: 0.04; Width: 0.88; height: 2.06
Doors_31 : Door Score: 0.543852769110514 => Referencelevel: 0.02; Width: 0.67; height: 2.09
Doors_22 : Door Score: 0.6376453069759453 => Referencelevel: 0.03; Width: 0.97; height: 2.11
Doors_17 : Door Score: 0.5043595651180848 => Referencelevel: 0.04; Width: 0.57; height: 2.04
Doors_14 : Door Score: 0.6592198531005693 => Referencelevel: 0.01; Width: 1.16; height: 2.12
Doors_11 : Door Score: 0.6819457715490592 => Referencelevel: 0.03; Width: 1.17; height: 2.07
Doors_21 : Door Score: 0.6139765364709105 => Referencelevel: 0.01; Width: 1.1; height: 2.08
Doors_12 : Door Score: 0.6438205980217977 => Referencelevel: 0.02; Width: 0.92; height: 2.11
Doors_24 : Door Score: 0.7244203122802403 => Referencelevel: 0.01; Width: 0.86; height: 2.06
Doors_0 : Door Score: 0.7159302826549695 => Referencelevel: 0.01; Width: 0.85; height: 2.08
Doors_25 : Door Score: 0.7007998785765275 => Referencelevel: 0.03; Width:

Doors_41 : Door Score: 0.7546150289411132 => Referencelevel: 0.09; Width: 1.02; height: 2.03
Doors_40 : Door Score: 0.7025646692125697 => Referencelevel: 0.21; Width: 0.97; height: 2.01
Doors_39 : Door Score: 0.8018869585576265 => Referencelevel: 0.09; Width: 0.97; height: 2.0
Doors_60 : Door Score: 0.7457722679428431 => Referencelevel: 0.05; Width: 1.06; height: 2.04
Doors_62 : Door Score: 0.655834971471973 => Referencelevel: 0.06; Width: 0.97; height: 2.08
Doors_2 : Door Score: 0.7063323409142701 => Referencelevel: 0.02; Width: 0.99; height: 2.16
Doors_3 : Door Score: 0.651555550072504 => Referencelevel: 0.01; Width: 1.0; height: 2.12
Doors_52 : Door Score: 0.5864066168795461 => Referencelevel: 0.02; Width: 0.85; height: 2.11
Doors_57 : Door Score: 0.5660248786988467 => Referencelevel: 0.02; Width: 1.13; height: 2.19
Doors_12 : Door Score: 0.7072250378909317 => Referencelevel: 0.06; Width: 0.97; height: 2.06
Doors_13 : Door Score: 0.7104070299086365 => Referencelevel: 0.06; Width: 0.

Doors_0 : Door Score: 0.7018359653846078 => Referencelevel: 0.03; Width: 0.95; height: 2.09
Doors_10 : Door Score: 0.5659288412591684 => Referencelevel: 0.01; Width: 1.16; height: 2.16
Doors_35 : Door Score: 0.5449184272729835 => Referencelevel: 0.02; Width: 1.13; height: 2.15
Doors_7 : Door Score: 0.6274956706554993 => Referencelevel: 0.02; Width: 1.09; height: 2.08
Doors_36 : Door Score: 0.5480392221652943 => Referencelevel: 0.02; Width: 1.08; height: 2.17
Doors_14 : Door Score: 0.6925814968347552 => Referencelevel: 0.04; Width: 1.14; height: 2.03
Doors_4 : Door Score: 0.711743655567584 => Referencelevel: 0.04; Width: 0.98; height: 2.05
Doors_17 : Door Score: 0.6418501005094983 => Referencelevel: 0.03; Width: 1.12; height: 2.04
Doors_18 : Door Score: 0.5899172393394554 => Referencelevel: 0.03; Width: 1.05; height: 2.15
Doors_20 : Door Score: 0.6696346370292747 => Referencelevel: 0.03; Width: 1.1; height: 2.05
Doors_11 : Door Score: 0.6476650260842365 => Referencelevel: 0.02; Width: 1

Doors_16 : Door Score: 0.5789894267398377 => Referencelevel: 0.02; Width: 0.93; height: 2.14
Doors_70 : Door Score: 0.6823829673684163 => Referencelevel: 0.02; Width: 1.04; height: 2.07
Doors_33 : Door Score: 0.540684784832208 => Referencelevel: 0.02; Width: 1.18; height: 2.19
Doors_52 : Door Score: 0.687472999640133 => Referencelevel: 0.04; Width: 0.94; height: 2.04
Doors_50 : Door Score: 0.6882761296500333 => Referencelevel: 0.02; Width: 0.94; height: 2.03
Doors_10 : Door Score: 0.6218733181901599 => Referencelevel: 0.02; Width: 1.07; height: 2.14
Doors_49 : Door Score: 0.7488077795505526 => Referencelevel: 0.02; Width: 0.99; height: 2.09
Doors_34 : Door Score: 0.6111797765286071 => Referencelevel: 0.01; Width: 0.97; height: 2.14
Doors_73 : Door Score: 0.5789737935429033 => Referencelevel: 0.02; Width: 1.01; height: 2.19
Doors_39 : Door Score: 0.6353253956203877 => Referencelevel: 0.02; Width: 0.89; height: 2.13
Doors_48 : Door Score: 0.5418778212951578 => Referencelevel: 0.02; Width

Doors_8 : Door Score: 0.7179698565731878 => Referencelevel: 0.0; Width: 1.01; height: 1.97
Doors_3 : Door Score: 0.5856343354608701 => Referencelevel: 0.01; Width: 1.15; height: 2.1
Doors_13 : Door Score: 0.7576629561963287 => Referencelevel: 0.01; Width: 1.06; height: 1.98
Doors_0 : Door Score: 0.6606818645933399 => Referencelevel: 0.01; Width: 0.7; height: 1.98
Number of doors: 4
JSON data written to file: /home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t8/test/25_Parking_01_F1_doors.json
final text_encoder_type: bert-base-uncased
Model loaded from /home/sdegeyter/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])
/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t4/test/34_Parking_04_F1_small_pred.laz
/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t6/test/34_Parking_04_F1_walls.ttl
proces

Doors_5 : Door Score: 0.640062119623889 => Referencelevel: 0.01; Width: 0.91; height: 2.06
Doors_3 : Door Score: 0.7533429845001387 => Referencelevel: 0.02; Width: 0.88; height: 2.08
Doors_2 : Door Score: 0.5524114944105561 => Referencelevel: 0.01; Width: 1.13; height: 2.14
Number of doors: 3
JSON data written to file: /home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t8/test/34_Parking_04_F1_doors.json
